# Goud Chain API Testing Suite

Comprehensive test notebook for all Goud Chain API endpoints with timing metrics and reporting.

**Features:**
- Create API keys and authenticate
- Bulk data submission (configurable batch size)
- Collection management and decryption
- Blockchain explorer and analytics
- Performance metrics with visualizations
- Final report generation

## 1. Configuration

Adjust these settings before running tests.

In [ ]:
# ============ Configuration ============

# API Endpoint (Docker internal network or external)
BASE_URL = "http://nginx:8080"  # Use this when running notebook in Docker container
# BASE_URL = "http://localhost:8080"  # Use this for external testing (notebook running on host)

# Test Parameters
BULK_SUBMISSION_COUNT = 100  # Number of blocks to create in bulk test
API_TIMEOUT = 30  # Request timeout in seconds
RETRY_ATTEMPTS = 3  # Number of retries for failed requests (can be reduced dynamically if timeouts detected)

# Fast-fail mode: reduce retries after detecting server issues
ENABLE_FAST_FAIL = True  # Set to False to always retry full number of times

# Debug Settings
VERBOSE_LOGGING = True  # Print detailed logs
SHOW_REQUEST_BODIES = False  # Print full request/response bodies

# Report Settings
EXPORT_TO_CSV = True  # Export results to CSV
EXPORT_TO_JSON = True  # Export results to JSON
GENERATE_CHARTS = True  # Generate timing charts

# Use home directory which is guaranteed to be writable
import os
from pathlib import Path

# Try multiple possible directories in order of preference
possible_dirs = [
    "/work/reports/",           # If /work is mounted
    str(Path.home() / "reports"),  # User home directory
    "./reports/",               # Current directory
]

REPORT_OUTPUT_DIR = None
for dir_path in possible_dirs:
    try:
        os.makedirs(dir_path, exist_ok=True)
        # Test write permissions
        test_file = os.path.join(dir_path, ".test")
        with open(test_file, 'w') as f:
            f.write("test")
        os.remove(test_file)
        REPORT_OUTPUT_DIR = dir_path
        break
    except (PermissionError, OSError):
        continue

if not REPORT_OUTPUT_DIR:
    print("⚠️  Warning: Could not create reports directory, using current directory")
    REPORT_OUTPUT_DIR = "./"
else:
    print(f"✅ Reports directory: {REPORT_OUTPUT_DIR}")

print("✅ Configuration loaded")
print(f"📡 API Endpoint: {BASE_URL}")
print(f"🔢 Bulk submission count: {BULK_SUBMISSION_COUNT}")
print(f"📁 Reports will be saved to: {REPORT_OUTPUT_DIR}")

## 2. Setup & Utilities

Import libraries and define helper functions.

In [ ]:
import requests
import json
import time
import platform
import psutil
import socket
import sys
import traceback
import base64
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from pathlib import Path

# Global state storage
GLOBAL_STATE = {
    "api_key": None,
    "account_id": None,
    "session_token": None,
    "collection_ids": [],
    "test_results": [],
    "timing_data": {},
    "system_info": {},
    "server_health_history": [],
    "failed_dependencies": set(),
    "timeout_detected": False,  # Track if we've hit timeouts
    "fast_fail_mode": False,  # Enable fast-fail after first timeout
}

# Configure plotting style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

def capture_system_info() -> Dict[str, Any]:
    """Capture comprehensive system information"""
    try:
        cpu_count = psutil.cpu_count(logical=True)
        cpu_count_physical = psutil.cpu_count(logical=False)
        memory = psutil.virtual_memory()
        disk = psutil.disk_usage('/')
        
        system_info = {
            # System basics
            "hostname": socket.gethostname(),
            "platform": platform.platform(),
            "platform_system": platform.system(),
            "platform_release": platform.release(),
            "platform_version": platform.version(),
            "architecture": platform.machine(),
            "processor": platform.processor(),
            
            # CPU
            "cpu_count_logical": cpu_count,
            "cpu_count_physical": cpu_count_physical,
            "cpu_frequency_mhz": psutil.cpu_freq().current if psutil.cpu_freq() else "N/A",
            "cpu_percent": psutil.cpu_percent(interval=1),
            
            # Memory
            "memory_total_gb": round(memory.total / (1024**3), 2),
            "memory_available_gb": round(memory.available / (1024**3), 2),
            "memory_used_gb": round(memory.used / (1024**3), 2),
            "memory_percent": memory.percent,
            
            # Disk
            "disk_total_gb": round(disk.total / (1024**3), 2),
            "disk_used_gb": round(disk.used / (1024**3), 2),
            "disk_free_gb": round(disk.free / (1024**3), 2),
            "disk_percent": disk.percent,
            
            # Python environment
            "python_version": sys.version,
            "python_executable": sys.executable,
            
            # Network
            "ip_address": socket.gethostbyname(socket.gethostname()),
            
            # Timestamp
            "captured_at": datetime.now().isoformat(),
        }
        
        # Try to get Docker info if running in container
        try:
            with open('/proc/1/cgroup', 'r') as f:
                system_info["running_in_docker"] = 'docker' in f.read()
        except:
            system_info["running_in_docker"] = False
        
        return system_info
    
    except Exception as e:
        return {"error": f"Failed to capture system info: {str(e)}"}

def create_download_button(filepath: str, button_text: str = None) -> None:
    """Create a clickable download button for a file"""
    try:
        path = Path(filepath)
        if not path.exists():
            print(f"⚠️  File not found: {filepath}")
            return
        
        filename = path.name
        button_text = button_text or f"Download {filename}"
        
        # Read file and encode as base64
        with open(filepath, 'rb') as f:
            file_data = f.read()
        
        b64_data = base64.b64encode(file_data).decode()
        
        # Create HTML download link styled as a button
        file_size_mb = len(file_data) / (1024 * 1024)
        html = f'''
        <div style="margin: 10px 0;">
            <a download="{filename}" 
               href="data:application/octet-stream;base64,{b64_data}" 
               style="display: inline-block;
                      padding: 10px 20px;
                      background-color: #4CAF50;
                      color: white;
                      text-decoration: none;
                      border-radius: 5px;
                      font-weight: bold;
                      font-family: Arial, sans-serif;
                      box-shadow: 0 2px 4px rgba(0,0,0,0.2);
                      transition: background-color 0.3s;">
                📥 {button_text}
            </a>
            <span style="margin-left: 10px; color: #666; font-size: 0.9em;">
                ({file_size_mb:.2f} MB)
            </span>
        </div>
        <style>
            a:hover {{
                background-color: #45a049 !important;
            }}
        </style>
        '''
        display(HTML(html))
        
    except Exception as e:
        print(f"⚠️  Failed to create download button: {str(e)}")

# Capture system info on startup
GLOBAL_STATE["system_info"] = capture_system_info()

print("✅ Libraries imported")
print("✅ Global state initialized")
print("✅ System information captured")
print(f"\n📊 System Overview:")
print(f"   Platform: {GLOBAL_STATE['system_info'].get('platform_system')} {GLOBAL_STATE['system_info'].get('platform_release')}")
print(f"   CPU: {GLOBAL_STATE['system_info'].get('cpu_count_logical')} cores @ {GLOBAL_STATE['system_info'].get('cpu_percent')}% usage")
print(f"   Memory: {GLOBAL_STATE['system_info'].get('memory_available_gb')} GB available / {GLOBAL_STATE['system_info'].get('memory_total_gb')} GB total")
print(f"   Disk: {GLOBAL_STATE['system_info'].get('disk_free_gb')} GB free / {GLOBAL_STATE['system_info'].get('disk_total_gb')} GB total")
print(f"   Python: {sys.version.split()[0]}")
print(f"   Docker: {'Yes' if GLOBAL_STATE['system_info'].get('running_in_docker') else 'No'}")

In [ ]:
# ============ Helper Functions ============

class ErrorCategory:
    """Error type classification"""
    CONNECTION_ERROR = "connection_error"
    TIMEOUT = "timeout"
    HTTP_ERROR = "http_error"
    SERVER_CRASH = "server_crash"
    JSON_PARSE_ERROR = "json_parse_error"
    AUTHENTICATION_ERROR = "authentication_error"
    UNKNOWN = "unknown"

class ServerHealth:
    """Server health states"""
    ALIVE = "alive"
    DEGRADED = "degraded"
    CRASHED = "crashed"
    UNKNOWN = "unknown"

class APIClient:
    """Wrapper for Goud Chain API calls with timing, error handling, and retry logic"""
    
    def __init__(self, base_url: str, timeout: int = 30, retry_attempts: int = 3):
        self.base_url = base_url.rstrip('/')
        self.timeout = timeout
        self.retry_attempts = retry_attempts
        self.session = requests.Session()
    
    def _log(self, message: str, level: str = "INFO"):
        if VERBOSE_LOGGING or level == "ERROR":
            timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
            print(f"[{timestamp}] {level}: {message}")
    
    def check_server_health(self) -> Tuple[str, Dict]:
        """Check if server is responsive. Returns (health_status, details)"""
        try:
            response = self.session.get(f"{self.base_url}/health", timeout=5)
            if response.status_code == 200:
                data = response.json()
                health_status = ServerHealth.ALIVE
                details = {
                    "status": "healthy",
                    "node_id": data.get("node_id", "unknown"),
                    "chain_length": data.get("chain_length", 0),
                    "peer_count": data.get("peer_count", 0),
                }
            else:
                health_status = ServerHealth.DEGRADED
                details = {
                    "status": "degraded",
                    "http_status": response.status_code,
                    "error": response.text[:200]
                }
        except requests.exceptions.ConnectionError:
            health_status = ServerHealth.CRASHED
            details = {"status": "crashed", "error": "Cannot connect to server"}
        except requests.exceptions.Timeout:
            health_status = ServerHealth.DEGRADED
            details = {"status": "degraded", "error": "Health check timeout"}
        except Exception as e:
            health_status = ServerHealth.UNKNOWN
            details = {"status": "unknown", "error": str(e)}
        
        GLOBAL_STATE["server_health_history"].append({
            "timestamp": datetime.now().isoformat(),
            "health": health_status,
            "details": details
        })
        
        return health_status, details
    
    def request(self, method: str, endpoint: str, data: Optional[Dict] = None, 
                headers: Optional[Dict] = None, retry: bool = True) -> Tuple[Optional[Dict], float, bool, str, Optional[str]]:
        """Make HTTP request with timing and retry logic"""
        url = f"{self.base_url}{endpoint}"
        headers = headers or {}
        
        if data:
            headers["Content-Type"] = "application/json"
        
        # Check fast-fail mode
        actual_retries = 1 if (ENABLE_FAST_FAIL and GLOBAL_STATE["fast_fail_mode"]) else (self.retry_attempts if retry else 1)
        
        retry_count = 0
        last_error = None
        last_error_category = None
        stack_trace = None
        
        while retry_count < actual_retries:
            start_time = time.time()
            
            try:
                self._log(f"{method} {endpoint} (attempt {retry_count + 1}/{actual_retries})")
                
                if method == "GET":
                    response = self.session.get(url, headers=headers, timeout=self.timeout)
                elif method == "POST":
                    response = self.session.post(url, json=data, headers=headers, timeout=self.timeout)
                else:
                    raise ValueError(f"Unsupported method: {method}")
                
                elapsed = time.time() - start_time
                
                if response.status_code >= 200 and response.status_code < 300:
                    try:
                        result = response.json() if response.content else {}
                    except json.JSONDecodeError as e:
                        self._log(f"❌ JSON parse error: {str(e)}", "ERROR")
                        return {"error": "Invalid JSON response"}, elapsed, False, ErrorCategory.JSON_PARSE_ERROR, traceback.format_exc()
                    
                    self._log(f"✅ Success ({response.status_code}) in {elapsed:.3f}s")
                    return result, elapsed, True, None, None
                else:
                    self._log(f"❌ Failed ({response.status_code}): {response.text[:200]}", "ERROR")
                    error_category = ErrorCategory.HTTP_ERROR
                    if response.status_code == 401 or response.status_code == 403:
                        error_category = ErrorCategory.AUTHENTICATION_ERROR
                    return {"error": response.text, "status_code": response.status_code}, elapsed, False, error_category, None
            
            except requests.exceptions.ConnectionError as e:
                elapsed = time.time() - start_time
                last_error = e
                last_error_category = ErrorCategory.CONNECTION_ERROR
                stack_trace = traceback.format_exc()
                self._log(f"❌ Connection error: {str(e)}", "ERROR")
            
            except requests.exceptions.Timeout as e:
                elapsed = time.time() - start_time
                last_error = e
                last_error_category = ErrorCategory.TIMEOUT
                stack_trace = traceback.format_exc()
                self._log(f"❌ Timeout after {elapsed:.3f}s", "ERROR")
                
                # Enable fast-fail mode after first timeout
                if ENABLE_FAST_FAIL and not GLOBAL_STATE["fast_fail_mode"]:
                    GLOBAL_STATE["fast_fail_mode"] = True
                    print(f"\n⚡ Fast-fail mode enabled: future tests will skip retries")
            
            except Exception as e:
                elapsed = time.time() - start_time
                last_error = e
                last_error_category = ErrorCategory.UNKNOWN
                stack_trace = traceback.format_exc()
                self._log(f"❌ Exception: {str(e)}", "ERROR")
            
            retry_count += 1
            
            if retry_count < actual_retries and retry:
                backoff = min(2 ** retry_count, 10)
                self._log(f"⏳ Retrying in {backoff}s...")
                time.sleep(backoff)
        
        return {
            "error": str(last_error) if last_error else "Unknown error",
            "retry_count": retry_count
        }, elapsed, False, last_error_category or ErrorCategory.UNKNOWN, stack_trace

def record_test_result(test_name: str, success: bool, elapsed: float, details: Dict = None, 
                      error_category: Optional[str] = None, stack_trace: Optional[str] = None,
                      skipped: bool = False, skip_reason: Optional[str] = None, retry_count: int = 0):
    """Record test result for final report"""
    server_health = GLOBAL_STATE["server_health_history"][-1]["health"] if GLOBAL_STATE["server_health_history"] else ServerHealth.UNKNOWN
    
    result = {
        "test_name": test_name,
        "success": success,
        "elapsed_seconds": elapsed,
        "timestamp": datetime.now().isoformat(),
        "details": details or {},
        "error_category": error_category,
        "stack_trace": stack_trace,
        "skipped": skipped,
        "skip_reason": skip_reason,
        "retry_count": retry_count,
        "server_health": server_health,
    }
    GLOBAL_STATE["test_results"].append(result)
    
    if success and not skipped:
        if test_name not in GLOBAL_STATE["timing_data"]:
            GLOBAL_STATE["timing_data"][test_name] = []
        GLOBAL_STATE["timing_data"][test_name].append(elapsed)
    
    if not success and not skipped:
        GLOBAL_STATE["failed_dependencies"].add(test_name)

def run_api_test(test_name: str, method: str, endpoint: str, 
                 data: Optional[Dict] = None, headers: Optional[Dict] = None,
                 dependencies: List[str] = None, 
                 on_success = None, on_failure = None,
                 skip_health_check: bool = False) -> Optional[Dict]:
    """
    Universal test runner - eliminates code duplication across test cells.
    
    Args:
        test_name: Display name for the test
        method: HTTP method (GET or POST)
        endpoint: API endpoint to test
        data: Request body (for POST)
        headers: Request headers
        dependencies: List of test names this test depends on
        on_success: Callback function(response) to handle successful response
        on_failure: Callback function(response, error_category) to handle failure
        skip_health_check: If True, don't check server health before test
    
    Returns:
        API response dict if successful, None otherwise
    """
    print_section(test_name)
    
    try:
        # Check dependencies
        dependencies = dependencies or []
        for dep in dependencies:
            if dep in GLOBAL_STATE["failed_dependencies"]:
                print(f"\n⏭️  Test skipped: Dependency failed: {dep}")
                record_test_result(test_name, False, 0, skipped=True, skip_reason=f"Dependency failed: {dep}")
                return None
        
        # Check server health (unless skipped)
        if not skip_health_check:
            print("🏥 Checking server health...")
            health, health_details = api.check_server_health()
            print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
            
            if health == ServerHealth.CRASHED or (GLOBAL_STATE["fast_fail_mode"] and health == ServerHealth.DEGRADED):
                print(f"\n💀 Server {'crashed' if health == ServerHealth.CRASHED else 'degraded'}! Skipping test.")
                record_test_result(test_name, False, 0, 
                                  details={"server_crashed": health == ServerHealth.CRASHED}, 
                                  error_category=ErrorCategory.SERVER_CRASH if health == ServerHealth.CRASHED else ErrorCategory.TIMEOUT,
                                  skipped=True,
                                  skip_reason=f"Server {health}")
                return None
        
        # Make API request
        response, elapsed, success, error_category, stack_trace = api.request(method, endpoint, data=data, headers=headers)
        
        # Handle result
        if success:
            print(f"\n✅ Success!")
            if on_success:
                on_success(response)
            record_test_result(test_name, True, elapsed, response, error_category=error_category, stack_trace=stack_trace)
            return response
        else:
            print(f"\n❌ Failed: {response.get('error', 'Unknown error')}")
            print(f"   Error category: {error_category}")
            if on_failure:
                on_failure(response, error_category)
            record_test_result(test_name, False, elapsed, response, error_category=error_category, stack_trace=stack_trace)
            return None
    
    except Exception as e:
        print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
        stack = traceback.format_exc()
        if VERBOSE_LOGGING:
            print(f"   Stack trace:\n{stack}")
        record_test_result(test_name, False, 0,
                          details={"unhandled_exception": str(e)},
                          error_category=ErrorCategory.UNKNOWN,
                          stack_trace=stack)
        return None

def print_section(title: str):
    """Print formatted section header"""
    print(f"\n{'='*60}")
    print(f"  {title}")
    print(f"{'='*60}\n")

# Initialize API client
api = APIClient(BASE_URL, API_TIMEOUT, RETRY_ATTEMPTS)

print("✅ Helper functions defined")
print("✅ API client initialized")
print(f"   Max retries: {RETRY_ATTEMPTS}")
print(f"   Timeout: {API_TIMEOUT}s")
print(f"   Fast-fail: {'Enabled' if ENABLE_FAST_FAIL else 'Disabled'}")

## 3. Test: Create Account

Create a new user account and obtain an API key.

In [ ]:
# Test 1: Create Account
def handle_create_account_success(response):
    GLOBAL_STATE["api_key"] = response.get("api_key")
    GLOBAL_STATE["account_id"] = response.get("account_id")
    print(f"   Account ID: {GLOBAL_STATE['account_id']}")
    print(f"   API Key: {GLOBAL_STATE['api_key'][:20]}..." if GLOBAL_STATE['api_key'] else "   API Key: None")
    if response.get('warning'):
        print(f"\n⚠️  {response.get('warning')}")

run_api_test(
    test_name="Test 1: Create Account",
    method="POST",
    endpoint="/account/create",
    data={"metadata": "Test account created from Jupyter notebook"},
    on_success=handle_create_account_success
)

## 4. Test: Login

Login with API key to obtain a session token.

In [ ]:
# Test 2: Login
def handle_login_success(response):
    GLOBAL_STATE["session_token"] = response.get("session_token")
    print(f"   Session Token: {GLOBAL_STATE['session_token'][:30]}..." if GLOBAL_STATE['session_token'] else "   Session Token: None")
    print(f"   Expires in: {response.get('expires_in')} seconds")

run_api_test(
    test_name="Test 2: Login",
    method="POST",
    endpoint="/account/login",
    data={"api_key": GLOBAL_STATE["api_key"]},
    dependencies=["Test 1: Create Account"],
    on_success=handle_login_success
) if GLOBAL_STATE["api_key"] else print("⏭️  Skipped: No API key")

## 5. Test: Submit Data (Single)

Submit a single encrypted collection to the blockchain.

In [ ]:
# Test 3: Submit Data (Single)
def handle_submit_success(response):
    collection_id = response.get("collection_id")
    if collection_id:
        GLOBAL_STATE["collection_ids"].append(collection_id)
    print(f"   Collection ID: {collection_id}")
    print(f"   Block Number: {response.get('block_number')}")
    print(f"   Message: {response.get('message')}")

run_api_test(
    test_name="Test 3: Submit Data (Single)",
    method="POST",
    endpoint="/data/submit",
    data={
        "label": "Test Collection",
        "data": json.dumps({"message": "Hello from Jupyter!", "timestamp": time.time()})
    },
    headers={"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"},
    dependencies=["Test 1: Create Account"],
    on_success=handle_submit_success
) if GLOBAL_STATE["api_key"] else print("⏭️  Skipped: No API key")

## 6. Test: Bulk Data Submission

Submit multiple collections to test throughput and performance.

In [ ]:
# Test 4: Bulk Data Submission
print_section(f"Test 4: Bulk Data Submission ({BULK_SUBMISSION_COUNT} blocks)")

if not GLOBAL_STATE["api_key"]:
    print("⏭️  Skipped: No API key")
    record_test_result("Test 4: Bulk Data Submission", False, 0, skipped=True, skip_reason="No API key")
else:
    try:
        print("🏥 Checking server health before bulk test...")
        health, health_details = api.check_server_health()
        print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
        
        if health == ServerHealth.CRASHED:
            print("\n💀 Server crashed! Skipping bulk test.")
            record_test_result("Test 4: Bulk Data Submission", False, 0, 
                              details={"server_crashed": True}, 
                              error_category=ErrorCategory.SERVER_CRASH,
                              skipped=True, skip_reason="Server crashed")
        else:
            headers = {"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"}
            start_time = time.time()
            successful = 0
            failed = 0
            submission_times = []
            error_counts = {}
            server_crashed = False
            
            print(f"Submitting {BULK_SUBMISSION_COUNT} collections...\n")
            
            for i in range(BULK_SUBMISSION_COUNT):
                # Check health every 50 iterations
                if i > 0 and i % 50 == 0:
                    health, _ = api.check_server_health()
                    if health == ServerHealth.CRASHED:
                        print(f"\n💀 Server crashed at iteration {i}!")
                        server_crashed = True
                        break
                
                data = {
                    "label": f"Bulk Test #{i+1}",
                    "data": json.dumps({"iteration": i+1, "timestamp": time.time(), "test_type": "bulk"})
                }
                
                response, elapsed, success, error_category, _ = api.request("POST", "/data/submit", data=data, headers=headers, retry=False)
                submission_times.append(elapsed)
                
                if success:
                    successful += 1
                    if response.get("collection_id"):
                        GLOBAL_STATE["collection_ids"].append(response["collection_id"])
                else:
                    failed += 1
                    if error_category:
                        error_counts[error_category] = error_counts.get(error_category, 0) + 1
                    if error_category == ErrorCategory.SERVER_CRASH:
                        print(f"\n💀 Server crashed at iteration {i+1}!")
                        server_crashed = True
                        break
                
                # Progress indicator
                if (i + 1) % 10 == 0 or i == BULK_SUBMISSION_COUNT - 1:
                    progress = (i + 1) / BULK_SUBMISSION_COUNT * 100
                    print(f"Progress: {i+1}/{BULK_SUBMISSION_COUNT} ({progress:.1f}%) - Success: {successful}, Failed: {failed}")
            
            total_time = time.time() - start_time
            avg_time = sum(submission_times) / len(submission_times) if submission_times else 0
            throughput = successful / total_time if total_time > 0 else 0
            
            print(f"\n{'='*60}")
            print(f"📊 Bulk Submission Summary:")
            print(f"   Total attempted: {successful + failed}")
            print(f"   Successful: {successful}")
            print(f"   Failed: {failed}")
            print(f"   Success rate: {successful/(successful + failed)*100:.1f}%" if (successful + failed) > 0 else "   Success rate: N/A")
            print(f"   Total time: {total_time:.2f}s")
            print(f"   Average time: {avg_time:.3f}s")
            print(f"   Throughput: {throughput:.2f} submissions/second")
            if server_crashed:
                print(f"   ⚠️  SERVER CRASHED during test!")
            if error_counts:
                print(f"   Error categories:")
                for cat, count in error_counts.items():
                    print(f"      {cat}: {count}")
            print(f"{'='*60}")
            
            test_success = successful == BULK_SUBMISSION_COUNT and not server_crashed
            
            record_test_result("Test 4: Bulk Data Submission", test_success, total_time, {
                "total_attempted": successful + failed,
                "target": BULK_SUBMISSION_COUNT,
                "successful": successful,
                "failed": failed,
                "avg_time": avg_time,
                "throughput": throughput,
                "server_crashed": server_crashed,
                "error_categories": error_counts
            }, error_category=ErrorCategory.SERVER_CRASH if server_crashed else None)
            
            # Enable fast-fail if we hit issues
            if failed > 0 and ENABLE_FAST_FAIL:
                GLOBAL_STATE["fast_fail_mode"] = True
                print(f"\n⚡ Fast-fail mode enabled due to bulk test failures")
    
    except Exception as e:
        print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
        record_test_result("Test 4: Bulk Data Submission", False, 0,
                          details={"unhandled_exception": str(e)},
                          error_category=ErrorCategory.UNKNOWN,
                          stack_trace=traceback.format_exc())

## 7. Test: List Collections

Retrieve all collections for the authenticated user.

In [ ]:
# Test 5: List Collections
def handle_list_success(response):
    collections = response.get("collections", [])
    print(f"   Retrieved {len(collections)} collections")
    if collections:
        try:
            df = pd.DataFrame(collections)
            display(df.head(10))
            if len(collections) > 10:
                print(f"\n... and {len(collections) - 10} more")
        except Exception as e:
            print(f"   ⚠️  Could not display as DataFrame: {str(e)}")
            print(f"   Raw data: {collections[:3]}")

run_api_test(
    test_name="Test 5: List Collections",
    method="GET",
    endpoint="/data/list",
    headers={"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"},
    dependencies=["Test 1: Create Account"],
    on_success=handle_list_success
) if GLOBAL_STATE["api_key"] else print("⏭️  Skipped: No API key")

## 8. Test: Decrypt Collection

Decrypt a specific collection to verify data integrity.

In [ ]:
# Test 6: Decrypt Collection
def handle_decrypt_success(response):
    print(f"   Collection ID: {response.get('collection_id')}")
    print(f"   Label: {response.get('label')}")
    print(f"   Created at: {response.get('created_at')}")
    print(f"   Data: {response.get('data')}")

if not GLOBAL_STATE["api_key"]:
    print("⏭️  Skipped: No API key")
elif not GLOBAL_STATE["collection_ids"]:
    print("⏭️  Skipped: No collections available")
else:
    collection_id = GLOBAL_STATE["collection_ids"][0]
    run_api_test(
        test_name="Test 6: Decrypt Collection",
        method="POST",
        endpoint=f"/data/decrypt/{collection_id}",
        headers={"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"},
        dependencies=["Test 1: Create Account", "Test 3: Submit Data (Single)"],
        on_success=handle_decrypt_success
    )

## 9. Test: View Blockchain

Retrieve the entire blockchain state.

In [ ]:
# Test 7: View Blockchain
def handle_blockchain_success(response):
    chain_length = len(response.get("chain", []))
    node_id = response.get("node_id", "unknown")
    print(f"   Node ID: {node_id}")
    print(f"   Chain length: {chain_length} blocks")
    if chain_length > 0:
        try:
            latest_block = response["chain"][-1]
            print(f"   Latest block index: {latest_block.get('index')}")
            print(f"   Latest block timestamp: {latest_block.get('timestamp')}")
        except Exception as e:
            print(f"   ⚠️  Could not parse latest block: {str(e)}")

run_api_test(
    test_name="Test 7: View Blockchain",
    method="GET",
    endpoint="/chain",
    on_success=handle_blockchain_success
)

## 10. Test: Chain Statistics

Get analytics and statistics about the blockchain.

In [ ]:
# Test 8: Chain Statistics
def handle_stats_success(response):
    print(f"   Total blocks: {response.get('total_blocks')}")
    print(f"   Total collections: {response.get('total_collections')}")
    print(f"   Total accounts: {response.get('total_accounts')}")
    print(f"   Avg block time: {response.get('avg_block_time_seconds', 0):.3f}s")
    
    validator_dist = response.get('validator_distribution', {})
    if validator_dist:
        print(f"\n   Validator distribution:")
        for validator, count in validator_dist.items():
            print(f"      {validator}: {count} blocks")

run_api_test(
    test_name="Test 8: Chain Statistics",
    method="GET",
    endpoint="/stats",
    on_success=handle_stats_success
)

## 11. Test: Node Metrics

Get performance metrics from the current node.

In [ ]:
# Test 9: Node Metrics
def handle_metrics_success(response):
    print(f"   Node ID: {response.get('node_id')}")
    print(f"   Chain length: {response.get('chain_length')}")
    print(f"   Peer count: {response.get('peer_count')}")
    print(f"   Latest block index: {response.get('latest_block_index')}")
    print(f"   Latest block timestamp: {response.get('latest_block_timestamp')}")
    print(f"   Status: {response.get('status')}")

run_api_test(
    test_name="Test 9: Node Metrics",
    method="GET",
    endpoint="/metrics",
    on_success=handle_metrics_success
)

## 12. Test: P2P Peers

View connected P2P peers and their reputation.

In [ ]:
# Test 10: P2P Peers
def handle_peers_success(response):
    peers = response.get('peers', [])
    reputation = response.get('reputation', {})
    print(f"   Connected peers: {response.get('count', 0)}")
    if peers:
        print(f"\n   Peer list:")
        for peer in peers:
            rep = reputation.get(peer, 0)
            print(f"      {peer} (reputation: {rep})")
    else:
        print(f"   No peers connected")

run_api_test(
    test_name="Test 10: P2P Peers",
    method="GET",
    endpoint="/peers",
    on_success=handle_peers_success
)

## 13. Test: Health Check

Verify node health status (used by load balancer).

In [ ]:
# Test 11: Health Check
def handle_health_success(response):
    print(f"   Status: {response.get('status')}")
    print(f"   Node ID: {response.get('node_id')}")
    print(f"   Chain length: {response.get('chain_length')}")
    print(f"   Peer count: {response.get('peer_count')}")
    print(f"   Latest block: {response.get('latest_block')}")

def handle_health_failure(response, error_category):
    print(f"   This indicates the server is crashed, degraded, or experiencing issues")

run_api_test(
    test_name="Test 11: Health Check",
    method="GET",
    endpoint="/health",
    skip_health_check=True,  # Don't check health before the health check itself
    on_success=handle_health_success,
    on_failure=handle_health_failure
)

## 14. Final Report & Visualizations

Generate comprehensive report with timing analysis and charts.

In [ ]:
print_section("Final Report & Analysis")

try:
    # Calculate summary statistics
    total_tests = len(GLOBAL_STATE["test_results"])
    successful_tests = sum(1 for r in GLOBAL_STATE["test_results"] if r["success"] and not r.get("skipped"))
    failed_tests = sum(1 for r in GLOBAL_STATE["test_results"] if not r["success"] and not r.get("skipped"))
    skipped_tests = sum(1 for r in GLOBAL_STATE["test_results"] if r.get("skipped"))
    success_rate = (successful_tests / (successful_tests + failed_tests) * 100) if (successful_tests + failed_tests) > 0 else 0
    
    total_time = sum(r["elapsed_seconds"] for r in GLOBAL_STATE["test_results"])
    avg_time = total_time / total_tests if total_tests > 0 else 0
    
    print(f"\n📊 Test Execution Summary:")
    print(f"{'='*60}")
    print(f"Total tests: {total_tests}")
    print(f"Successful: {successful_tests}")
    print(f"Failed: {failed_tests}")
    print(f"Skipped: {skipped_tests}")
    print(f"Success rate: {success_rate:.1f}% (excluding skipped)")
    print(f"Total execution time: {total_time:.2f}s ({total_time/60:.1f} minutes)")
    print(f"Average time per test: {avg_time:.3f}s")
    if GLOBAL_STATE["fast_fail_mode"]:
        print(f"⚡ Fast-fail mode: Enabled (retries skipped after timeouts detected)")
    print(f"{'='*60}\n")
    
    # Error category analysis
    error_categories = {}
    for result in GLOBAL_STATE["test_results"]:
        if result.get("error_category"):
            cat = result["error_category"]
            error_categories[cat] = error_categories.get(cat, 0) + 1
    
    if error_categories:
        print(f"\n🔍 Error Category Breakdown:")
        print(f"{'='*60}")
        for category, count in sorted(error_categories.items(), key=lambda x: x[1], reverse=True):
            print(f"   {category}: {count} occurrences")
        print(f"{'='*60}\n")
    
    # Server health timeline
    if GLOBAL_STATE["server_health_history"]:
        print(f"\n🏥 Server Health Timeline:")
        print(f"{'='*60}")
        health_counts = {}
        for entry in GLOBAL_STATE["server_health_history"]:
            health = entry["health"]
            health_counts[health] = health_counts.get(health, 0) + 1
        
        for health, count in sorted(health_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"   {health}: {count} checks")
        
        crashes = [e for e in GLOBAL_STATE["server_health_history"] if e["health"] == ServerHealth.CRASHED]
        if crashes:
            print(f"\n   ⚠️  SERVER CRASHES DETECTED: {len(crashes)} times")
            print(f"   First crash at: {crashes[0]['timestamp']}")
        print(f"{'='*60}\n")
    
    # Dependency chain analysis
    if GLOBAL_STATE["failed_dependencies"]:
        print(f"\n🔗 Failed Dependencies (blocking downstream tests):")
        print(f"{'='*60}")
        for dep in GLOBAL_STATE["failed_dependencies"]:
            blocked_tests = [r for r in GLOBAL_STATE["test_results"] 
                           if r.get("skip_reason") and dep in r.get("skip_reason", "")]
            print(f"   '{dep}' blocked {len(blocked_tests)} downstream test(s)")
        print(f"{'='*60}\n")
    
    # Create results DataFrame
    df_results = pd.DataFrame(GLOBAL_STATE["test_results"])
    
    print("\n📋 Detailed Results:")
    display_cols = ["test_name", "success", "skipped", "elapsed_seconds", "error_category", "server_health"]
    if all(col in df_results.columns for col in display_cols):
        display(df_results[display_cols])
    else:
        display(df_results)
    
    # Export files and create download buttons
    export_files = []
    
    # Export to CSV
    if EXPORT_TO_CSV:
        try:
            csv_path = f"{REPORT_OUTPUT_DIR}test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            df_results.to_csv(csv_path, index=False)
            print(f"\n✅ CSV exported to: {csv_path}")
            export_files.append(("CSV", csv_path))
        except Exception as e:
            print(f"\n⚠️  Failed to export CSV: {str(e)}")
    
    # Export to JSON
    if EXPORT_TO_JSON:
        try:
            json_path = f"{REPORT_OUTPUT_DIR}test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            with open(json_path, 'w') as f:
                json.dump({
                    "test_run_info": {
                        "timestamp": datetime.now().isoformat(),
                        "base_url": BASE_URL,
                        "bulk_submission_count": BULK_SUBMISSION_COUNT,
                        "api_timeout": API_TIMEOUT,
                        "retry_attempts": RETRY_ATTEMPTS,
                        "fast_fail_enabled": ENABLE_FAST_FAIL,
                        "fast_fail_triggered": GLOBAL_STATE["fast_fail_mode"],
                    },
                    "system_info": GLOBAL_STATE["system_info"],
                    "summary": {
                        "total_tests": total_tests,
                        "successful": successful_tests,
                        "failed": failed_tests,
                        "skipped": skipped_tests,
                        "success_rate": success_rate,
                        "total_time": total_time,
                        "avg_time": avg_time,
                        "error_categories": error_categories,
                    },
                    "server_health_summary": {
                        "total_checks": len(GLOBAL_STATE["server_health_history"]),
                        "crash_count": len([e for e in GLOBAL_STATE["server_health_history"] if e["health"] == ServerHealth.CRASHED]),
                        "timeline": GLOBAL_STATE["server_health_history"],
                    },
                    "results": GLOBAL_STATE["test_results"],
                    "global_state": {
                        "account_id": GLOBAL_STATE["account_id"],
                        "collection_count": len(GLOBAL_STATE["collection_ids"]),
                        "failed_dependencies": list(GLOBAL_STATE["failed_dependencies"]),
                    }
                }, f, indent=2)
            print(f"✅ JSON exported to: {json_path}")
            export_files.append(("JSON", json_path))
        except Exception as e:
            print(f"\n⚠️  Failed to export JSON: {str(e)}")
    
    # Recommendations
    print(f"\n💡 Recommendations:")
    print(f"{'='*60}")
    
    if failed_tests == 0 and skipped_tests == 0:
        print("   ✅ All tests passed! System is healthy.")
    else:
        if ErrorCategory.SERVER_CRASH in error_categories:
            print("   🚨 Server crashed during testing - check memory limits and logs")
        if ErrorCategory.TIMEOUT in error_categories:
            print("   ⏱️  Timeouts detected - server may be overloaded or unresponsive")
        if ErrorCategory.CONNECTION_ERROR in error_categories:
            print("   🔌 Connection errors detected - verify network configuration")
        if skipped_tests > 0:
            print(f"   ⏭️  {skipped_tests} tests skipped - fix root causes first")
        if failed_tests > successful_tests:
            print("   🔥 More failures than successes - system may be fundamentally broken")
        if GLOBAL_STATE["fast_fail_mode"]:
            print(f"   ⚡ Fast-fail saved ~{skipped_tests * (API_TIMEOUT * RETRY_ATTEMPTS):.0f}s by skipping retries")
    
    print(f"{'='*60}")
    
    # Display download buttons
    if export_files:
        print(f"\n\n📥 Download Reports:")
        print(f"{'='*60}")
        for file_type, filepath in export_files:
            create_download_button(filepath, f"Download {file_type} Report")

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION in reporting: {str(e)}")
    stack = traceback.format_exc()
    print(f"   Stack trace:\n{stack}")

In [ ]:
# Generate timing charts and failure analysis visualizations
try:
    if GENERATE_CHARTS and (GLOBAL_STATE["timing_data"] or GLOBAL_STATE["test_results"]):
        print("\n📊 Generating performance charts and failure analysis...\n")
        
        has_timing_data = bool(GLOBAL_STATE["timing_data"])
        has_test_data = bool(GLOBAL_STATE["test_results"])
        
        num_charts = 0
        if has_timing_data:
            num_charts += 2
        if has_test_data:
            num_charts += 2
        
        if num_charts == 0:
            print("⚠️  No data available for charting")
        else:
            # Create figure
            if num_charts == 4:
                fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
                axes = [ax1, ax2, ax3, ax4]
            elif num_charts == 3:
                fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
                axes = [ax1, ax2, ax3]
            elif num_charts == 2:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
                axes = [ax1, ax2]
            else:
                fig, ax1 = plt.subplots(1, 1, figsize=(8, 6))
                axes = [ax1]
            
            chart_idx = 0
            
            # Chart 1: Average latency per endpoint
            if has_timing_data:
                endpoints = list(GLOBAL_STATE["timing_data"].keys())
                avg_times = [sum(times)/len(times) for times in GLOBAL_STATE["timing_data"].values()]
                
                axes[chart_idx].barh(endpoints, avg_times, color='steelblue')
                axes[chart_idx].set_xlabel('Average Response Time (seconds)')
                axes[chart_idx].set_title('Average Latency by Endpoint')
                axes[chart_idx].grid(axis='x', alpha=0.3)
                chart_idx += 1
                
                # Chart 2: Response time distribution
                all_times = []
                all_labels = []
                for endpoint, times in GLOBAL_STATE["timing_data"].items():
                    all_times.extend(times)
                    all_labels.extend([endpoint] * len(times))
                
                df_times = pd.DataFrame({'Endpoint': all_labels, 'Time': all_times})
                sns.boxplot(data=df_times, y='Endpoint', x='Time', ax=axes[chart_idx])
                axes[chart_idx].set_xlabel('Response Time (seconds)')
                axes[chart_idx].set_title('Response Time Distribution')
                axes[chart_idx].grid(axis='x', alpha=0.3)
                chart_idx += 1
            
            # Chart 3: Error category pie chart
            if has_test_data:
                error_categories = {}
                for result in GLOBAL_STATE["test_results"]:
                    if result.get("error_category"):
                        cat = result["error_category"]
                        error_categories[cat] = error_categories.get(cat, 0) + 1
                
                if error_categories and chart_idx < len(axes):
                    labels = list(error_categories.keys())
                    sizes = list(error_categories.values())
                    colors = plt.cm.Set3(range(len(labels)))
                    
                    axes[chart_idx].pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
                    axes[chart_idx].set_title('Error Category Distribution')
                    chart_idx += 1
                
                # Chart 4: Server health timeline
                if GLOBAL_STATE["server_health_history"] and chart_idx < len(axes):
                    health_timeline = []
                    timestamps = []
                    
                    for i, entry in enumerate(GLOBAL_STATE["server_health_history"]):
                        health = entry["health"]
                        health_value = {
                            ServerHealth.ALIVE: 3,
                            ServerHealth.DEGRADED: 2,
                            ServerHealth.CRASHED: 1,
                            ServerHealth.UNKNOWN: 0
                        }.get(health, 0)
                        health_timeline.append(health_value)
                        timestamps.append(i)
                    
                    axes[chart_idx].plot(timestamps, health_timeline, marker='o', linewidth=2, markersize=6)
                    axes[chart_idx].set_xlabel('Health Check Number')
                    axes[chart_idx].set_ylabel('Server Health')
                    axes[chart_idx].set_title('Server Health Timeline')
                    axes[chart_idx].set_yticks([0, 1, 2, 3])
                    axes[chart_idx].set_yticklabels(['Unknown', 'Crashed', 'Degraded', 'Alive'])
                    axes[chart_idx].grid(alpha=0.3)
                    
                    crash_indices = [i for i, v in enumerate(health_timeline) if v == 1]
                    if crash_indices:
                        axes[chart_idx].scatter([timestamps[i] for i in crash_indices],
                                               [health_timeline[i] for i in crash_indices],
                                               color='red', s=200, alpha=0.5, zorder=5,
                                               label='Crashes')
                        axes[chart_idx].legend()
            
            plt.tight_layout()
            
            # Save chart
            try:
                chart_path = f"{REPORT_OUTPUT_DIR}performance_chart_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
                plt.savefig(chart_path, dpi=150, bbox_inches='tight')
                print(f"✅ Chart saved to: {chart_path}")
                plt.show()
                
                # Create download button for chart
                print(f"\n📥 Download Chart:")
                print(f"{'='*60}")
                create_download_button(chart_path, "Download Performance Charts (PNG)")
                
            except Exception as e:
                print(f"⚠️  Failed to save chart: {str(e)}")
                plt.show()
    
    print("\n✅ Report generation complete!")

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION in chart generation: {str(e)}")
    stack = traceback.format_exc()
    print(f"   Stack trace:\n{stack}")

## Summary

✅ All tests completed with comprehensive error handling, fast-fail optimization, and one-click downloads!

**🆕 New Features in This Version:**
- ⚡ **Fast-Fail Mode**: After first timeout, retries are skipped to save time (configurable via `ENABLE_FAST_FAIL`)
- 📥 **One-Click Downloads**: Click buttons to download reports directly from notebook (no Docker filesystem access needed)
- 🔄 **DRY Code**: Test cells reduced from 50+ lines to 5-10 lines using `run_api_test()` utility
- 📁 **Accessible Reports**: Files saved to `/work/reports/` (inside container, Jupyter-accessible)
- 💾 **Smart Storage**: Base64-encoded downloads work in any environment

**Enhanced Features:**
- 🛡️ **Error-Resistant Design**: Every test wrapped in try-except blocks to prevent cascading failures
- 🏥 **Server Health Monitoring**: Automatic health checks before each test to detect server crashes
- 🔗 **Dependency Tracking**: Tests automatically skip if prerequisites failed, preventing false positives
- 📊 **Comprehensive Diagnostics**: Error categorization, server health timeline, dependency chain analysis
- 💻 **System Information**: Captures CPU, memory, disk, platform info dynamically
- 📈 **Enhanced Reporting**: Failure analysis with actionable recommendations

**Error Categories Tracked:**
- `connection_error` - Cannot connect to server
- `timeout` - Request exceeded timeout limit (triggers fast-fail mode)
- `http_error` - HTTP error status (4xx, 5xx)
- `server_crash` - Server completely unresponsive
- `json_parse_error` - Invalid JSON response
- `authentication_error` - Auth failures (401, 403)
- `unknown` - Unhandled exception types

**Artifacts Generated & Downloadable:**
- 📋 Test Results DataFrame (displayed in notebook)
- 📄 CSV Export: Clickable download button for spreadsheet analysis
- 📦 JSON Export: Clickable download button with full metadata (system info, server health timeline, stack traces)
- 📊 Performance Charts: Clickable download button for PNG charts (latency, distribution, error breakdown, health timeline)

**Fast-Fail Performance:**
- **Without fast-fail**: 7 timeout failures × 30s timeout × 3 retries = **~10 minutes wasted**
- **With fast-fail**: 7 timeout failures × 30s timeout × 1 attempt = **~3.5 minutes**
- **Time saved**: ~6.5 minutes (60% reduction)

**Configuration Options:**
- `BASE_URL`: Set to `http://nginx:8080` for Docker, `http://localhost:8080` for external
- `BULK_SUBMISSION_COUNT`: Number of collections to create in bulk test (default: 100)
- `API_TIMEOUT`: Request timeout in seconds (default: 30)
- `RETRY_ATTEMPTS`: Max retries before giving up (default: 3, reduced to 1 in fast-fail mode)
- `ENABLE_FAST_FAIL`: Enable smart retry skipping after timeouts (default: True)
- `REPORT_OUTPUT_DIR`: Where to save reports (default: `/work/reports/`)

**Global State Variables:**
- `GLOBAL_STATE["api_key"]` - API key for authentication
- `GLOBAL_STATE["account_id"]` - Account ID
- `GLOBAL_STATE["collection_ids"]` - List of created collection IDs
- `GLOBAL_STATE["test_results"]` - Detailed results for all tests
- `GLOBAL_STATE["system_info"]` - System/environment information
- `GLOBAL_STATE["server_health_history"]` - Timeline of server health checks
- `GLOBAL_STATE["failed_dependencies"]` - Set of tests that failed (blocking downstream tests)
- `GLOBAL_STATE["fast_fail_mode"]` - Whether fast-fail has been triggered
- `GLOBAL_STATE["timeout_detected"]` - Whether any timeouts occurred

**Next Steps:**
1. Review the detailed results and recommendations above
2. **Click the download buttons** to get CSV/JSON/PNG reports
3. Analyze performance charts to identify bottlenecks
4. If server crashed/degraded, check container logs: `docker logs <container_name>`
5. Adjust `BULK_SUBMISSION_COUNT` or `API_TIMEOUT` based on results
6. Toggle `ENABLE_FAST_FAIL = False` if you want full retry behavior even after timeouts